Для выполнения данной работы предлагается использовать библиотеку [transformers](https://pypi.org/project/transformers/)

## Задачи:
  1. Получить векторное представление слов с помощью transformers
  2. Обучить модель для классификации текстов вакансий по роли (professional_roles)
  3. Обучить модель для определения размера оплаты труда (от и до или среднее значение) 


## Материалы:
  - https://github.com/huggingface/transformers/tree/master/notebooks
  - https://huggingface.co/models?language=ru&sort=downloads

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.9 MB/s eta 0:00:00


In [ ]:
import re
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask
  
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru") #Создается токенайзер
model = AutoModelForMaskedLM.from_pretrained("sberbank-ai/sbert_large_nlu_ru") #Создается модель трансформера

Some weights of BertForMaskedLM were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
encoded_input = tokenizer("Hello, world!", return_tensors='pt') #Формируются входные данные для трансформера
model_output = model(**encoded_input)
mean_pooling(model_output, encoded_input['attention_mask'])[0]

tensor([ 0.8412,  0.1372,  0.3569,  ...,  0.1967, -0.2601, -1.5904],
       grad_fn=<SelectBackward0>)

#0. Подготовка данных


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import torch
import numpy as np
import ast #Для преобразования строки в словарь
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [ ]:
vacancies_frame = pd.read_csv("vacancies_from_hh(72-107)++.csv", on_bad_lines='skip', sep='\t', nrows=1500)
vacancies_frame.drop(vacancies_frame.columns[:5], axis=1, inplace=True)

In [ ]:
vacancies_frame

,id,json,prepared_description,status
0,72323.0,"{'id': '72323', 'premium': False, 'billing_typ...","[['требование', 'обязательно', 'высокий', 'нез...",Запрос выполнен успешно
1,72325.0,"{'id': '72325', 'premium': False, 'billing_typ...","[['крупный', 'производственный', 'компания', '...",Запрос выполнен успешно
2,72326.0,"{'id': '72326', 'premium': False, 'billing_typ...","[['требование', 'знание', 'линейка', 'программ...",Запрос выполнен успешно
3,72327.0,"{'id': '72327', 'premium': False, 'billing_typ...","[['требование', 'знание', 'типовой', 'конфигур...",Запрос выполнен успешно
4,72329.0,"{'id': '72329', 'premium': False, 'billing_typ...","[['требование', 'знание', 'типовой', 'конфигур...",Запрос выполнен успешно
...,...,...,...,...
1495,82303.0,"{'id': '82303', 'premium': False, 'billing_typ...","[['крупный', 'металлоторговый', 'компания', 'п...",Запрос выполнен успешно
1496,82304.0,"{'id': '82304', 'premium': False, 'billing_typ...","[['обязанность', 'организация', 'развитие', 'д...",Запрос выполнен успешно
1497,82308.0,"{'id': '82308', 'premium': False, 'billing_typ...","[['компания', 'работать', 'область', 'корпорат...",Запрос выполнен успешно
1498,82314.0,"{'id': '82314', 'premium': False, 'billing_typ...","[['приглашаться', 'руководитель', 'медицинский...",Запрос выполнен успешно


In [ ]:
def check_vacancies_roles(frame):
  label_frame = pd.DataFrame({'professional_roles': [], 'id': []})
  for i, row in frame.iterrows():
    new_frame_line = {'professional_roles': ast.literal_eval(row['json'])['professional_roles'][0]['name'], 'id': ast.literal_eval(row['json'])['professional_roles'][0]['id']}
    label_frame = label_frame.append(new_frame_line, ignore_index=True)
  return label_frame

In [ ]:
vacancies_roles = check_vacancies_roles(vacancies_frame)

<ipython-input-6-752b838a1254>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label_frame = label_frame.append(new_frame_line, ignore_index=True)
<ipython-input-6-752b838a1254>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label_frame = label_frame.append(new_frame_line, ignore_index=True)
<ipython-input-6-752b838a1254>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label_frame = label_frame.append(new_frame_line, ignore_index=True)
<ipython-input-6-752b838a1254>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label_frame = label_frame.append(new_frame_line, ignore_index=True)
<ipython-input-6-752b838a1254>:5: FutureWarning: The

In [ ]:
vacancies_roles

,professional_roles,id
0,"Менеджер по продажам, менеджер по работе с кли...",70
1,Финансовый менеджер,137
2,Руководитель отдела продаж,106
3,Руководитель группы разработки,104
4,"Программист, разработчик",96
...,...,...
1495,Директор по маркетингу и PR (CMO),37
1496,Руководитель отдела продаж,106
1497,Экономист,142
1498,Медицинский представитель,65


In [ ]:
vacancies_set = vacancies_roles.drop_duplicates()

In [ ]:
vacancies_set

,professional_roles,id
0,"Менеджер по продажам, менеджер по работе с кли...",70
1,Финансовый менеджер,137
2,Руководитель отдела продаж,106
3,Руководитель группы разработки,104
4,"Программист, разработчик",96
...,...,...
990,Инженер по эксплуатации,46
1003,"Оператор call-центра, специалист контактного ц...",83
1040,"Повар, пекарь, кондитер",94
1118,Фармацевт-провизор,133


In [ ]:
def sort_vacancies_by_count_professional_roles(vacancies_roles, vacancies_set):
  roles_frame = pd.DataFrame({'professional_roles': [], 'count': [], 'id': []})
  for i, row in vacancies_set.iterrows():
    new_frame_line = {
        'professional_roles': row['professional_roles'], 
        'count': vacancies_roles[vacancies_roles['professional_roles'] == row['professional_roles']].shape[0],
        'id': row['id']}
    roles_frame = roles_frame.append(new_frame_line, ignore_index=True)
  return roles_frame.sort_values('count', ascending=False)



In [ ]:
roles_frame = sort_vacancies_by_count_professional_roles(vacancies_roles, vacancies_set)

<ipython-input-11-667bf089dfa7>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  roles_frame = roles_frame.append(new_frame_line, ignore_index=True)
<ipython-input-11-667bf089dfa7>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  roles_frame = roles_frame.append(new_frame_line, ignore_index=True)
<ipython-input-11-667bf089dfa7>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  roles_frame = roles_frame.append(new_frame_line, ignore_index=True)
<ipython-input-11-667bf089dfa7>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  roles_frame = roles_frame.append(new_frame_line, ignore_index=True)
<ipython-input-11-667bf089dfa7>:8: FutureWarning

In [ ]:
roles_frame.head(20)

,professional_roles,count,id
0,"Менеджер по продажам, менеджер по работе с кли...",186.0,70
9,Бухгалтер,154.0,18
13,Другое,150.0,40
6,"Секретарь, помощник руководителя, ассистент",83.0,110
19,Аудитор,56.0,16
31,Юрисконсульт,45.0,145
21,Менеджер по закупкам,37.0,66
16,Менеджер по персоналу,36.0,69
18,Директор по маркетингу и PR (CMO),34.0,37
22,"Менеджер по маркетингу, интернет-маркетолог",33.0,68


In [ ]:
def filter_vacancies_for_professional_id(frame):
  #137 - Финансовый менеджер
  #142 - Экономист
  #129 - Торговый представитель
  #106 - Руководитель отдела продаж
  #118 - Специалист по подбору персонала
  id_list = [137, 142, 129, 106, 118]
  for i, row in frame.iterrows():
    if int(ast.literal_eval(row['json'])['professional_roles'][0]['id']) not in id_list:
        frame = frame[frame.id != row['id']] # Удаляем строку из фрейма
  return frame

In [ ]:
def filter_vacancies_for_salary(frame):
  for index, row in frame.iterrows():
      if ast.literal_eval(row['json'])['salary'] == None: #Если зарплата для вакансии не указана в json
        frame = frame[frame.id != row['id']] #Удаляем ее
      elif (ast.literal_eval(row['json'])['salary']['currency'] != 'USD'): #Аналогично, если зарплата указана не в USD
        frame = frame[frame.id != row['id']]
      elif (ast.literal_eval(row['json'])['salary']['to'] == None): #Аналогично, если средняя сумма не указана
        frame = frame[frame.id != row['id']]
  return frame

In [ ]:
vacancies_frame = filter_vacancies_for_professional_id(vacancies_frame)

In [ ]:
vacancies_frame = filter_vacancies_for_salary(vacancies_frame)

In [ ]:
vacancies_frame

,id,json,prepared_description,status
1,72325.0,"{'id': '72325', 'premium': False, 'billing_typ...","[['крупный', 'производственный', 'компания', '...",Запрос выполнен успешно
11,72355.0,"{'id': '72355', 'premium': False, 'billing_typ...","[['требование', 'кандидат', 'высокий', 'образо...",Запрос выполнен успешно
18,72383.0,"{'id': '72383', 'premium': False, 'billing_typ...","[['требование', 'год', 'образование', 'средний...",Запрос выполнен успешно
26,72449.0,"{'id': '72449', 'premium': False, 'billing_typ...","[['приглашаться', 'директор', 'экономика', 'фи...",Запрос выполнен успешно
44,72604.0,"{'id': '72604', 'premium': False, 'billing_typ...","[['требование', 'положительный', 'опыт', 'подб...",Запрос выполнен успешно
...,...,...,...,...
1407,81810.0,"{'id': '81810', 'premium': False, 'billing_typ...","[['крупный', 'парфюмерный', 'компания', 'требо...",Запрос выполнен успешно
1408,81811.0,"{'id': '81811', 'premium': False, 'billing_typ...","[['крупный', 'парфюмерный', 'компания', 'требо...",Запрос выполнен успешно
1433,81976.0,"{'id': '81976', 'premium': False, 'billing_typ...","[['фармацевтический', 'компания', 'приглашатьс...",Запрос выполнен успешно
1481,82230.0,"{'id': '82230', 'premium': False, 'billing_typ...","[['крупный', 'парфюмерный', 'компания', 'требо...",Запрос выполнен успешно


#1. Получение векторного представления описаний вакансий с помощью transformers

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask
  
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") #Создается токенайзер
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased") #Создается предобученная модель трансформера

In [ ]:
encoded_input = tokenizer("Hello, world!", return_tensors='pt') #Формируются входные данные для трансформера
model_output = model(**encoded_input)
mean_pooling(model_output, encoded_input['attention_mask'])[0]

tensor([ -9.8838, -10.0403, -10.0316,  ...,  -8.5659,  -8.4822,  -5.5498],
       grad_fn=<SelectBackward0>)

In [ ]:
def get_tensors(frame): #Функция для формирования вектора из строки фрейма
	tensors = []
	for i, row in frame.iterrows():
		encoded_input = tokenizer(row['descriptions'],  return_tensors='pt')
		model_output = model(**encoded_input)
		tensors.append(mean_pooling(model_output, encoded_input['attention_mask'])[0])
	return np.array(tensors, dtype='f')

In [ ]:
def remove_garbage(raw_text): #Функция для удаление HTML тегов (мусора)
  regex, text = r"<[ /a-zA-Z]*>", raw_text
  return re.sub(regex, "", raw_text)

In [ ]:
def get_descriptions_from_frame(frame): #Функция для получения описаний от каждой вакансии
  descriptions = []
  for i, row in frame.iterrows():
    descriptions.append(remove_garbage(ast.literal_eval(row['json'])['description']))
  return descriptions

In [ ]:
descriptions = get_descriptions_from_frame(vacancies_frame)

In [ ]:
descriptions

['В крупную производственную компанию приглашается Финансовый директор. Основные обязанности: работа с кредитными организациями, проектное финансирование, планирование финансовых потоков, разработка и анализ финансовых показателей деятельности предприятия,постановка управленческой и финансовой отчетности.Требования: 25-45 лет, высшее образование, опыт работы в аналогичной должности - от 3-5 лет.Место работы: г.Самара.',
 'Требования к кандидатам: высшее образование, опыт работы в данном направлении (банковский аудит) не менее 3 лет. Знания и практические навыки проведения проверок по следующим направлениям: РКО, кредитование, ценные бумаги, налогообложение, валютный контроль и валютное обслуживание, пластиковые карты, документарные операции. Знание соответствующих нормативных актов, правил совершения и оформления выше перечисленных банковских операций. Желательно знание английского языка, МСФО. Пользователь ПК. Аналитический склад ума, ответственность, высокая работоспособность, способ

In [ ]:
encoded_input = tokenizer(descriptions, padding=True, truncation=True, max_length=128, return_tensors='pt')

In [ ]:
model_output = model(**encoded_input)

In [ ]:
tensors = mean_pooling(model_output, encoded_input['attention_mask'])

In [ ]:
tensors.shape

30522

In [ ]:
X = tensors.detach().numpy()

In [ ]:
X

array([[-4.049366 , -4.27547  , -4.0528874, ..., -3.1977525, -3.481874 ,
        -2.760034 ],
       [-4.3902426, -4.617679 , -4.3942637, ..., -3.5314596, -3.9429243,
        -3.2543273],
       [-4.3953743, -4.6498137, -4.406225 , ..., -3.270988 , -3.880203 ,
        -2.960131 ],
       ...,
       [-3.9638395, -4.202756 , -3.9788249, ..., -2.983764 , -3.5568466,
        -2.8759727],
       [-3.9544692, -4.130628 , -3.9787436, ..., -3.042423 , -3.739084 ,
        -2.357502 ],
       [-4.0869446, -4.383889 , -4.108018 , ..., -3.263246 , -3.9599104,
        -2.8019733]], dtype=float32)

#2. Классификация текстов вакансий по роли (professional_roles)

##Выделение целевого признака

In [ ]:
def get_labels(frame): #Функция для получения целевого признака (профессиональной роли)
  label_frame = pd.DataFrame({'professional_roles': []})
  for i, row in frame.iterrows():
    new_frame_line = {'professional_roles': ast.literal_eval(row['json'])['professional_roles'][0]['name']}
    label_frame = label_frame.append(new_frame_line, ignore_index=True)
  return label_frame

In [ ]:
y = get_labels(vacancies_frame)

<ipython-input-32-6719fc6b23be>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label_frame = label_frame.append(new_frame_line, ignore_index=True)
<ipython-input-32-6719fc6b23be>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label_frame = label_frame.append(new_frame_line, ignore_index=True)
<ipython-input-32-6719fc6b23be>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label_frame = label_frame.append(new_frame_line, ignore_index=True)
<ipython-input-32-6719fc6b23be>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label_frame = label_frame.append(new_frame_line, ignore_index=True)
<ipython-input-32-6719fc6b23be>:5: FutureWarning

In [ ]:
y

,professional_roles
0,Финансовый менеджер
1,Экономист
2,Торговый представитель
3,Финансовый менеджер
4,Специалист по подбору персонала
...,...
88,Торговый представитель
89,Торговый представитель
90,Руководитель отдела продаж
91,Торговый представитель


##Деление на train/test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=99, test_size=0.2, shuffle=True) 

##Логистическая регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
logreg_clf = LogisticRegression()

In [ ]:
logreg_clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
LR_prediction = logreg_clf.predict(X_test)

In [ ]:
LR_prediction

array(['Экономист', 'Экономист', 'Руководитель отдела продаж',
       'Торговый представитель', 'Специалист по подбору персонала',
       'Финансовый менеджер', 'Торговый представитель', 'Экономист',
       'Торговый представитель', 'Финансовый менеджер',
       'Специалист по подбору персонала', 'Экономист',
       'Руководитель отдела продаж', 'Экономист', 'Финансовый менеджер',
       'Экономист', 'Экономист', 'Экономист', 'Торговый представитель'],
      dtype=object)

In [ ]:
y_test

,professional_roles
42,Экономист
34,Специалист по подбору персонала
54,Руководитель отдела продаж
53,Торговый представитель
80,Руководитель отдела продаж
25,Финансовый менеджер
61,Торговый представитель
56,Экономист
28,Руководитель отдела продаж
7,Экономист


In [ ]:
print(accuracy_score(LR_prediction, y_test))

0.42105263157894735


#3. Модель для определения среднего размера оплаты труда

##Выделение целевого признака

In [ ]:
import ast #Для преобразования строки в словарь

def get_salarys(frame):
  salarys_frame = pd.DataFrame({'salarys': []})
  for i, row in frame.iterrows():
    new_frame_line = {'salarys': ast.literal_eval(row['json'])['salary']['to']}
    salarys_frame = salarys_frame.append(new_frame_line, ignore_index=True)
  return salarys_frame

In [ ]:
salarys_frame = get_salarys(vacancies_frame)

<ipython-input-44-dc521fa9f484>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  salarys_frame = salarys_frame.append(new_frame_line, ignore_index=True)
<ipython-input-44-dc521fa9f484>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  salarys_frame = salarys_frame.append(new_frame_line, ignore_index=True)
<ipython-input-44-dc521fa9f484>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  salarys_frame = salarys_frame.append(new_frame_line, ignore_index=True)
<ipython-input-44-dc521fa9f484>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  salarys_frame = salarys_frame.append(new_frame_line, ignore_index=True)
<ipython-input-44-dc521fa9f484>:

In [ ]:
salarys_frame

,salarys
0,5000.0
1,1700.0
2,700.0
3,5000.0
4,1500.0
...,...
88,800.0
89,800.0
90,1500.0
91,800.0


##Деление на train/test

In [ ]:
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X, salarys_frame, random_state=99, test_size=0.2, shuffle=True) 

##Регрессия Лассо


In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.linear_model import LassoCV

In [ ]:
RF_reg1  = LassoCV()

In [ ]:
RF_reg1.fit(X_reg_train, y_reg_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.246e+04, tolerance: 9.898e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.151e+04, tolerance: 9.898e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.

LassoCV()

In [ ]:
RF_predicted = RF_reg1.predict(X_reg_test)

In [ ]:
print('МАЕ: {}'.format(mean_absolute_error(y_reg_test, RF_predicted)))
print('RMSE: {}'.format(mean_squared_error(y_reg_test, RF_predicted, squared=False)))

МАЕ: 554.466552734375
RMSE: 650.6972643905317
